In [1]:
import os
import argparse
import numpy as np
import tensorflow as tf

from gat_lstm import STDN
from sklearn.metrics import mean_squared_error

D:\app2018\Anaconda35\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\app2018\Anaconda35\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\app2018\Anaconda35\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\app2018\Anaconda35\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1)

In [ ]:
def test(model, data_generator, sess, saver):
    train_inputs, train_labels = data_generator.get_all_data(purpose='train')
    train_inputs, train_labels = train_inputs[0], train_labels[0]

    test_inputs, test_labels = data_generator.get_all_data(purpose='test')
    test_inputs, test_labels = test_inputs[0], test_labels[0]

    train_batch_num = int(train_inputs.shape[1] / update_batch_size)
    test_batch_num = int(test_inputs.shape[1] / update_batch_size)

    if len(output_dir) > 0:
        data_generator.save_test_ground_truth(output_dir=output_dir,
                                              test_data_num=test_batch_num * update_batch_size)
    for epoch in range(epochs):
        total_test_loss = []
        total_outputa = []

        for i in range(test_batch_num):
            inputa = test_inputs[:, i * update_batch_size: (i+1) * update_batch_size, :, :]
            labela = test_labels[:, i * update_batch_size: (i+1) * update_batch_size, :]
            if "att" in model_type:
                dummy_clusters = np.zeros(shape=(len(inputa), update_batch_size, 1))
                feed_dict = {model.inputa: inputa, model.labela: labela, model.clustera: dummy_clusters}
            else:
                feed_dict = {model.inputa: inputa, model.labela: labela}
            outputa, loss1, = sess.run([model.outputas, model.total_loss1], feed_dict)
            total_outputa.append(outputa)
            total_test_loss.append(loss1)
        total_outputa = np.concatenate(total_outputa, axis=1)

        if len(output_dir) > 0:
            np.savez(output_dir + "/output_" + model_type, total_outputa)
            saver.save(sess, output_dir + "/model_" + model_type)
        print(epoch, np.sqrt(np.mean(total_test_loss)))

        total_train_loss = [] 
        total_train_outputa = []

        for i in range(train_batch_num):
            inputa = train_inputs[:, i * update_batch_size: (i + 1) * update_batch_size, :, :]
            labela = train_labels[:, i * update_batch_size: (i + 1) * update_batch_size, :]
            if "att" in model_type:
                dummy_clusters = np.zeros(shape=(len(inputa), update_batch_size, 1))
                feed_dict = {model.inputa: inputa, model.labela: labela, model.clustera: dummy_clusters}
            else:
                feed_dict = {model.inputa: inputa, model.labela: labela}
            sess.run([model.finetune_op], feed_dict)
            outputa, loss1 = sess.run([model.outputas, model.total_loss1], feed_dict)
            total_train_outputa.append(outputa)
            total_train_loss.append(loss1)

        if len(output_dir) > 0:
            np.savez(output_dir + "/output_train_" + model_type, total_train_outputa)
        print("train:", epoch, np.sqrt(np.mean(total_train_loss)))
